# Indexed Expressions: Representing and manipulating tensors, pseudotensors, etc. in NRPy+

### NRPy+ Source Code for this module: [indexedexp.py](../edit/indexedexp.py)

NRPy+ contains functionality for dealing with indexed expressions. To enable the basic functionality, first import three modules

In [1]:
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri

## Rank-1 Indexed Expressions

Indexed expressions of rank 1 are stored as [Python lists](https://www.tutorialspoint.com/python/python_lists.htm). 

There are two ways to declare indexed expressions:
+ **Symbolically:** $\texttt{declarerank1}$("basename",*optional: DIM*)
+ **Initialize to zero:** $\texttt{zerorank1}$(*optional: DIM*) $\leftarrow$ As we will see below, initializing to zero is useful if the indexed expression depends entirely on some other indexed or non-indexed expressions.

For example, the 3-vector $\beta^i$ (upper index denotes contravariant) can be declared in NRPy+ symbolically as follows:

In [2]:
# Set the dimension to 3
par.set_parval_from_str("DIM",3)

# Declare rank-1 contravariant ("U") vector
betaU = ixp.declarerank1("betaU")

# Print the result. It's a list!
print(betaU)

[betaU0, betaU1, betaU2]


NRPy+'s standard naming convention for indexed rank-1 expressions is 
+ **\[base variable name\]+\["U" for contravariant (up index) or "D" for covariant (down index)\]**

After declaring the vector, $\texttt{betaU0}$, $\texttt{betaU1}$, and $\texttt{betaU2}$ can only be accessed or manipulated through list access; i.e., via $\texttt{betaU[0]}$, $\texttt{betaU[1]}$, and $\texttt{betaU[2]}$, respectively. 

For example, let's multiply $\texttt{betaU1}$ by 2:

In [3]:
betaU[1] *= 2
print("The 3-vector betaU is now set to "+str(betaU))
print("The component betaU[1] is now set to "+str(betaU[1]))

The 3-vector betaU is now set to [betaU0, 2*betaU1, betaU2]
The component betaU[1] is now set to 2*betaU1


Alternatively, we could simply initialize $\text{betaU[i]}$ to zero, and then use :

In [4]:
# Initialize betaU to zero
betaU = ixp.zerorank1()
# Prove it:
print("The 3-vector betaU is initialized to zero: "+str(betaU))

print("\n\nNext we will set betaU to [0,1,2]:")

# Get the dimension we just set, so we know how many indices to loop over
DIM = par.parval_from_str("DIM")

for i in range(DIM):
    betaU[i] = i
    
print("The 3-vector betaU is now set to: "+str(betaU))


The 3-vector betaU is initialized to zero: [0, 0, 0]


Next we will set betaU to [0,1,2]:
The 3-vector betaU is now set to: [0, 1, 2]


### Next, let's declare the variable $\beta_j$ and perform the dot product $\beta^i \beta_i$:

In [5]:
# First set betaU back to its initial value
betaU = ixp.declarerank1("betaU")

# Declare beta_j:
betaD = ixp.declarerank1("betaD")

# Get the dimension we just set, so we know how many indices to loop over
DIM = par.parval_from_str("DIM")

# Initialize dot product to zero
dotprod = 0

# Perform dot product beta^i beta_i
for i in range(DIM):
    dotprod += betaU[i]*betaD[i]

# Print result!
print(dotprod)

betaD0*betaU0 + betaD1*betaU1 + betaD2*betaU2


## Rank-2 and Higher Indexed Expressions

Moving to higher ranks, rank-2 indexed expressions are stored as lists of lists, rank-3 indexed expressions as lists of lists of lists, etc. For example

+ the covariant rank-2 tensor $g_{ij}$ is declared as $\texttt{gDD[i][j]}$ in NRPy+, so that e.g., $\texttt{gDD[0][2]}$ is stored with name "gDD02" and
+ the rank-2 tensor $T^{\mu}{}_{\nu}$ is declared as $\texttt{TUD[m][n]}$ in NRPy+ (index names are of course arbitrary).

*Caveat*: Note that it is currently up to the user to determine whether the combination of indexed expressions makes sense; NRPy+ does not track whether up and down indices are written consistently.

NRPy+ supports symmetries in indexed expressions (above rank 1), so that if $h_{ij} = h_{ji}$, then declaring $\texttt{hDD[i][j]}$ to be symmetric in NRPy+ will result in both $\texttt{hDD[0][2]}$ and $\texttt{hDD[2][0]}$ mapping to the *single* SymPy variable $hDD02$.

To see how this works in NRPy+, let's define in NRPy+ a symmetric, rank-2 tensor $h_{ij}$ in three dimensions, and then compute the contraction, which should be given by $$con = h^{ij}h_{ij} = h_{00} h^{00} + h_{11} h^{11} + h_{22} h^{22} + 2 (h_{01} h^{01} + h_{02} h^{02} + h_{12} h^{12}).$$

In [6]:
# Get the dimension we just set (should be set to 3).
DIM = par.parval_from_str("DIM")

# Declare h_{ij}=hDD[i][j] and h^{ij}=hUU[i][j]
hUU = ixp.declarerank2("hUU","sym12")
hDD = ixp.declarerank2("hDD","sym12")

# Perform sum h^{ij} h_{ij}, initializing contraction result to zero
con = 0
for i in range(DIM):
    for j in range(DIM):
        con += hUU[i][j]*hDD[i][j]

# Print result
print(con)

hDD00*hUU00 + 2*hDD01*hUU01 + 2*hDD02*hUU02 + hDD11*hUU11 + 2*hDD12*hUU12 + hDD22*hUU22


### Next let's create the C code for the contraction variable $\text{con}$, without CSE (common subexpression elimination)

In [7]:
from outputC import *

outputC(con,"con")

/*
 *  Original SymPy expression:
 *  "con = hDD00*hUU00 + 2*hDD01*hUU01 + 2*hDD02*hUU02 + hDD11*hUU11 + 2*hDD12*hUU12 + hDD22*hUU22"
 */
{
   con = hDD00*hUU00 + 2*hDD01*hUU01 + 2*hDD02*hUU02 + hDD11*hUU11 + 2*hDD12*hUU12 + hDD22*hUU22;
}



### Finally, let's see how it looks with SIMD support enabled

In [8]:
par.set_parval_from_str("SIMD_enable",True)
outputC(con,"con")

/*
 *  Original SymPy expression:
 *  "con = hDD00*hUU00 + 2*hDD01*hUU01 + 2*hDD02*hUU02 + hDD11*hUU11 + 2*hDD12*hUU12 + hDD22*hUU22"
 */
{
   const double tmp_Integer_2 = 2.000000000000000000000000000000000;
   const REAL_SIMD_ARRAY  = Set1SIMD(tmp_Integer_2);

   con = FusedMulAddSIMD(hUU02, MulSIMD(hDD02, _Integer_2), FusedMulAddSIMD(hUU01, MulSIMD(hDD01, _Integer_2), FusedMulAddSIMD(hDD22, hUU22, FusedMulAddSIMD(hDD11, hUU11, FusedMulAddSIMD(hUU12, MulSIMD(hDD12, _Integer_2), MulSIMD(hDD00, hUU00))))));
}



## Exercise: Write the NRPy+ code required to generate the needed C code for the lowering operator: $g_{ij} \beta^i$, and set the result to C variables "$\texttt{betaD0out}$", "$\texttt{betaD1out}$", and "$\texttt{betaD2out}$" [solution](Tutorial-Indexed_Expressions-soln.ipynb). *Hint: You will want to use the zerorank1() function*

## To complete this exercise, you must first reset all variables in the notebook:

In [9]:
# *Uncomment* the below %reset command and then press <Shift>+<Enter>. 
#    Respond with "y" in the dialog box to reset all variables.
# %reset

## Write your solution below: